
# Measuring Fast Withdrawal Contract Storage Costs:
- This notebook allows you to deploy the Fast Withdrawal contract along with the Ticket Router Tester contract to check whether reducing information in the withdrawal status leads to any significant storage costs.

In [1]:
from typing import Optional
from scripts.helpers.ticket_content import TicketContent
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.contracts import XtzTicketer
from scripts.helpers.addressable import Addressable

def make_xtz_withdrawal(
    withdrawer: Addressable,
    amount: int = 1_000_000,
    withdrawal_id: int = 0,
    timestamp: Optional[int] = None
):

    discounted_amount = amount
    full_amount = amount
    timestamp = timestamp or withdrawer.now()

    return Withdrawal(
        withdrawal_id=withdrawal_id,
        full_amount=full_amount,
        ticketer=XTZ_TICKETER_ADDRESS,
        content=TicketContent(
            token_id=0,
            token_info=None,
        ),
        timestamp=timestamp,
        base_withdrawer=get_address(withdrawer),
        payload=pack(discounted_amount, 'nat'),
        l2_caller=bytes(20)
    )

def get_paid_storage_size_diff(result) -> int:
    """Returns the paid storage size difference for the `payout_operation` result."""
    metadata = result[0]['contents'][0]['metadata']
    return int(metadata['operation_result']['paid_storage_size_diff'])

def print_cost_reduction_diff(new_result, base_result):
    new_paid_storage = get_paid_storage_size_diff(new_result)
    base_paid_storage = get_paid_storage_size_diff(base_result)
    reduction = base_paid_storage - new_paid_storage
    xtz_reduction = reduction * 250 / 10**6

    click.echo('-- Calculating Cost Reduction')
    click.echo('Paid storage for new base payout: ' + wrap(accent(base_paid_storage)) + ' bytes')
    click.echo('Paid storage for new new payout: ' + wrap(accent(new_paid_storage)) + ' bytes')
    click.echo('Reduction: ' + wrap(accent(reduction)) + ' bytes')
    click.echo('Equivalent to: ' + wrap(accent(xtz_reduction)) + ' XTZ')

def finalize_withdrawal(tezos_account, tester, withdrawal):
    # Minting ticket:
    xtz_ticketer = XtzTicketer.from_address(tezos_account, XTZ_TICKETER_ADDRESS)
    ticket = xtz_ticketer.read_ticket(tezos_account)

    opg = xtz_ticketer.mint(tezos_account, amount).send()
    tezos_account.wait(opg)
    ticket = xtz_ticketer.read_ticket(tezos_account)

    # Sending settle_withdrawal operation:
    opg = tezos_account.bulk(
        tester.set_settle_withdrawal(
            target=fast_withdrawal,
            withdrawal=withdrawal,
        ),
        ticket.transfer(tester),
    ).send()
    result = tezos_account.wait(opg)
    return result

In [2]:
from docs.scenarios.setup import *

_, _, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `231.987803 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.240557203 ꜩ`


## Deploy TicketRouterTester:
- This contract will serve as a mock implementation of the Smart Rollup.

In [3]:
from scripts.helpers.contracts import TicketRouterTester

'''
opg = TicketRouterTester.originate(tezos_account).send()
tezos_account.wait(opg)
tester = TicketRouterTester.from_opg(tezos_account, opg)
click.echo(
    'Successfully deployed Ticket Router Tester, address: '
    + wrap(accent(tester.address))
)
'''
tester = TicketRouterTester.from_address(tezos_account, 'KT1FoycxLiTeB6E8SPi6P3AcDZ5Usor4QJ29')
tester.address

'KT1FoycxLiTeB6E8SPi6P3AcDZ5Usor4QJ29'

## Deploy FastWithdrawal:

In [4]:
from scripts.tezos import deploy_fast_withdrawal

fast_withdrawal = deploy_fast_withdrawal.callback(
    xtz_ticketer_address=XTZ_TICKETER_ADDRESS,
    smart_rollup_address=get_address(tester),
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Native XTZ Ticketer address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `KT1FoycxLiTeB6E8SPi6P3AcDZ5Usor4QJ29`
Successfully deployed Fast Withdrawal, address: `KT1RQ6wynZhsWF849z6wnSXrrUNEw722nad9`


## Purchasing withdrawal on Tezos side:

In [5]:
amount = 1_000_000
base_withdrawal = make_xtz_withdrawal(tezos_account, amount)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=base_withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=amount,
).send()
base_result = tezos_account.wait(opg)

base_paid_storage = get_paid_storage_size_diff(base_result)
click.echo('Paid storage for new base payout: ' + wrap(accent(base_paid_storage)) + ' bytes')

Paid storage for new base payout: `94` bytes


## Cementing withdrawal with Ticket Router Tester call:

In [6]:
result = finalize_withdrawal(tezos_account, tester, base_withdrawal)

## Adding new record just after cementation
- This operation should be cheaper if cementing freed some storage:

In [7]:
withdrawal_a = make_xtz_withdrawal(tezos_account, amount)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal_a,
    service_provider=get_address(tezos_account),
    xtz_amount=amount,
).send()
new_result = tezos_account.wait(opg)
print_cost_reduction_diff(new_result, base_result)

-- Calculating Cost Reduction
Paid storage for new base payout: `94` bytes
Paid storage for new new payout: `69` bytes
Reduction: `25` bytes
Equivalent to: `0.00625` XTZ


## Ensuring Another Withdrawal Still Requires Full Storage Payment:
- Since no cementation operation has been executed to reduce the contract’s storage, the next withdrawal should require payment for the full storage difference:

In [8]:
withdrawal_b = make_xtz_withdrawal(tezos_account, amount)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal_b,
    service_provider=get_address(tezos_account),
    xtz_amount=amount,
).send()
new_result = tezos_account.wait(opg)
print_cost_reduction_diff(new_result, base_result)

-- Calculating Cost Reduction
Paid storage for new base payout: `94` bytes
Paid storage for new new payout: `94` bytes
Reduction: `0` bytes
Equivalent to: `0.0` XTZ


## And now cementing both of the withdrawals and checking the price of the new added payout:

In [9]:
result_a = finalize_withdrawal(tezos_account, tester, withdrawal_a)
result_a[0]['hash']

'oo95wDZQnnXjsYCGDHYfmKmpgztvppsRFA9ifgdorQ6D4TUiE4L'

In [10]:
result_b = finalize_withdrawal(tezos_account, tester, withdrawal_b)
result_b[0]['hash']

'onw8rrDprbGDHZHQgJChXpFHdiSZqZDZueKmMPHr4Z41XbA44vj'

In [11]:
withdrawal_c = make_xtz_withdrawal(tezos_account, amount)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal_c,
    service_provider=get_address(tezos_account),
    xtz_amount=amount,
).send()
new_result = tezos_account.wait(opg)
print_cost_reduction_diff(new_result, base_result)

-- Calculating Cost Reduction
Paid storage for new base payout: `94` bytes
Paid storage for new new payout: `44` bytes
Reduction: `50` bytes
Equivalent to: `0.0125` XTZ
